In [65]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

In [66]:
####### Parameters #########

BATCH_SIZE = 64
EPOCHS = 10
VAL_SPLIT_RATIO = 0.1
NUM_CLASSES = 10
INPUT_SHAPE = (28, 28, 1)

In [67]:
####### Load Data ##########
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [68]:
######### Data Shape  #########
print('X_train: ' + str(x_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test:  '  + str(x_test.shape))
print('Y_test:  '  + str(y_test.shape))

X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [69]:
####### DATA PROCESSING ##########

# Normalization
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Reshape images
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

# Train/validation split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = VAL_SPLIT_RATIO)

In [70]:
####### MODEL ###########

model = keras.Sequential(
    [
        keras.Input(shape=INPUT_SHAPE),
        layers.Conv2D(32, kernel_size=3, activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(32, kernel_size=3, activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(32, kernel_size=5, strides=2, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Conv2D(64, kernel_size=3, activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=3, activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=5, strides=2, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Conv2D(128, kernel_size=4, activation="relu"),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dropout(0.4),
        layers.Dense(NUM_CLASSES, activation="softmax"),
    ]
)

In [71]:
######## Compile the model  ##########
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [72]:
####### CALLBACKS #########

# Set up learning rate scheduler
lr_sheduler = LearningRateScheduler(lambda x: 1.0e-3 * 0.95 ** x)

In [73]:
###### IMAGE GENERATOR WITH AUGMENTATION

datagen = ImageDataGenerator(
    rotation_range=10, zoom_range=0.10, width_shift_range=0.1, height_shift_range=0.1
)

In [75]:
####### TRAINING #########

history = model.fit(
    datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
    epochs=5,
    validation_data=(x_val, y_val),
    steps_per_epoch=x_train.shape[0] // BATCH_SIZE,
    callbacks=[lr_sheduler],
)

Epoch 1/5
843/843 [==============================] - 124s 147ms/step - loss: 0.2163 - accuracy: 0.9334 - val_loss: 0.0659 - val_accuracy: 0.9802 - lr: 0.0010
Epoch 2/5
843/843 [==============================] - 124s 147ms/step - loss: 0.1086 - accuracy: 0.9665 - val_loss: 0.0416 - val_accuracy: 0.9880 - lr: 9.5000e-04
Epoch 3/5
843/843 [==============================] - 124s 147ms/step - loss: 0.0854 - accuracy: 0.9752 - val_loss: 0.0287 - val_accuracy: 0.9912 - lr: 9.0250e-04
Epoch 4/5
843/843 [==============================] - 120s 142ms/step - loss: 0.0707 - accuracy: 0.9785 - val_loss: 0.0356 - val_accuracy: 0.9895 - lr: 8.5737e-04
Epoch 5/5
843/843 [==============================] - 127s 150ms/step - loss: 0.0656 - accuracy: 0.9801 - val_loss: 0.0221 - val_accuracy: 0.9935 - lr: 8.1451e-04


In [76]:
###### Predict test label from test data using train model
y_pred = model.predict(x_test)
y_pred[:5]

array([[1.2683046e-07, 1.2824311e-06, 3.5008128e-07, 2.9859544e-07,
        1.9683567e-07, 1.6314157e-09, 1.3343141e-09, 9.9999273e-01,
        7.1604731e-09, 4.8456313e-06],
       [4.8523216e-06, 1.2778166e-05, 9.9996388e-01, 6.1459245e-06,
        2.7997547e-07, 6.2212351e-08, 8.0421751e-06, 2.9859439e-06,
        9.8955877e-07, 4.2442295e-08],
       [5.1413385e-06, 9.9881613e-01, 3.5417233e-05, 2.2611453e-05,
        2.9669958e-04, 3.9104689e-05, 5.8222042e-05, 4.7207397e-04,
        7.9525053e-06, 2.4673028e-04],
       [9.9992871e-01, 6.2771943e-07, 2.0421569e-06, 2.9952517e-07,
        1.0666712e-06, 6.7959127e-07, 4.5279885e-06, 2.6373089e-06,
        1.5714261e-06, 5.7811907e-05],
       [6.2812924e-06, 5.0831813e-06, 1.4248944e-05, 4.1061872e-07,
        9.9926788e-01, 8.8192212e-07, 5.8201948e-07, 5.6210447e-06,
        3.0641431e-06, 6.9586409e-04]], dtype=float32)

In [77]:
score = model.evaluate(x_test, y_test,verbose=1)

print(score)

313/313 [==============================] - 4s 12ms/step - loss: 0.0236 - accuracy: 0.9918
[0.023561494424939156, 0.9918000102043152]
